In [1]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import cv2
import scipy.ndimage as ndimage
from scipy.ndimage import binary_dilation, maximum_filter, minimum_filter
from torchvision.models.inception import InceptionAux

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
df = pd.read_csv('/kaggle/input/processed-lungs/processed_lungs/labels.csv')

nodule_df = df[df['label'] == 1]
second_df = df[df['label'] == 0]

print("we print nodule ")
print(nodule_df)
print("now second one")
print(second_df)

we print nodule 
                   filename  label
0     00000004_000_lung.png      1
1     00000008_002_lung.png      1
2     00000013_025_lung.png      1
3     00000017_000_lung.png      1
4     00000021_000_lung.png      1
...                     ...    ...
6326  00030703_001_lung.png      1
6327  00030715_000_lung.png      1
6328  00030722_000_lung.png      1
6329  00030726_000_lung.png      1
6330  00030793_000_lung.png      1

[6331 rows x 2 columns]
now second one
                     filename  label
6331    00000001_000_lung.png      0
6332    00000001_001_lung.png      0
6333    00000001_002_lung.png      0
6334    00000002_000_lung.png      0
6335    00000003_000_lung.png      0
...                       ...    ...
112111  00030801_001_lung.png      0
112112  00030802_000_lung.png      0
112113  00030803_000_lung.png      0
112114  00030804_000_lung.png      0
112115  00030805_000_lung.png      0

[105785 rows x 2 columns]


In [4]:
df_bbox = pd.read_csv('/kaggle/input/bbox-list-2017-csv/BBox_List_2017.csv')

df_bbox = df_bbox[df_bbox["Finding Label"] == "Nodule"]
print(df_bbox)

          Image Index Finding Label     Bbox [x           y           w  \
668  00001688_000.png        Nodule  667.496296  276.317460  122.446561   
669  00004547_003.png        Nodule  899.386243  690.251852   53.096296   
670  00023078_000.png        Nodule  741.180952  687.001058   52.012698   
671  00023068_003.png        Nodule  351.085714  747.682540   35.758730   
672  00013911_000.png        Nodule  118.112169  483.284656   85.604233   
..                ...           ...         ...         ...         ...   
742  00013674_000.png        Nodule  210.217989  319.661376   58.514286   
743  00013751_003.png        Nodule  676.165079  610.065608   89.938624   
744  00010103_014.png        Nodule  343.500529  412.850794   41.176720   
745  00011576_000.png        Nodule  206.967196  582.975661   46.594709   
746  00030413_003.png        Nodule  255.729101  237.307937   88.855026   

             h]  Unnamed: 6  Unnamed: 7  Unnamed: 8  
668  150.620106         NaN         NaN      

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [5]:
BATCH_SIZE = 8
LR = 0.001
EPOCHS = 10

main_dest_dir = '/kaggle/working/'
source_base_dir = '/kaggle/input/processed-lungs'

class SegmentedChestXRayDataset(Dataset):
    def __init__(self, input_size):
        self.image_paths = []
        self.labels = []
        self.normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                     
        self.transform_positive = transforms.Compose([
            transforms.Resize(input_size),
            transforms.ToTensor(),
            self.normalize
        ])
        
        self.transform_negative = transforms.Compose([
            transforms.Resize(input_size),
            transforms.ToTensor(),
            self.normalize
        ])
        
        source_folder = 'processed_lungs/nodule'
        for _, row in df_bbox.iterrows():
            image_filename = row['Image Index']
            img_path = os.path.join(source_base_dir,
                                    source_folder,
                                    image_filename)
            root, ext = os.path.splitext(img_path)
            new_path = f"{root}_lung{ext}"
            self.image_paths.append(new_path)

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        image = Image.open(img_path).convert('RGB')
        
        image = self.transform_positive(image)
            
        return image, img_path

    def __len__(self):
        return len(self.image_paths)
        
    def tackle_idxs(self, idxs):
        image_paths_temp = []
        labels_temp = []
        
        for i in idxs:
            label = self.labels[i]
            img_path = self.image_paths[i]
            
            image_paths_temp.append(img_path)
            labels_temp.append(label)
        
        combined = list(zip(image_paths_temp, labels_temp))
        random.shuffle(combined)
        self.image_paths, self.labels = map(list, zip(*combined))

In [6]:
source_base_dir_data = '/kaggle/input/data/'

folder_ranges = [
    (1335, 6, 'images_001'),
    (3923, 13, 'images_002'),
    (6585, 6, 'images_003'),
    (9232, 3, 'images_004'),
    (11558, 7, 'images_005'),
    (13774, 25, 'images_006'),
    (16051, 9, 'images_007'),
    (18387, 34, 'images_008'),
    (20945, 49, 'images_009'),
    (24717, 0, 'images_010'),
    (28173, 2, 'images_011'),
    (30805, 0, 'images_012')
]

class ChestXRayDataset(Dataset):
    def __init__(self):
        self.image_paths = []
        self.labels = []
        
        for _, row in df_bbox.iterrows():
            image_filename = row['Image Index']
            x_cord = row['Bbox [x']
            y_cord = row['y']
            width_w = row['w']
            height_h = row['h]']
            base_name = os.path.splitext(image_filename)[0]
            part1_str, part2_str = base_name.split('_')
            part1, part2 = int(part1_str), int(part2_str)
    
            source_folder = None
            for f_part1, f_part2, f_name in folder_ranges:
                if part1 < f_part1 or (part1 == f_part1 and part2 <= f_part2):
                    source_folder = f_name
                    break
            if not source_folder:
                continue
    
            img_path = os.path.join(source_base_dir_data,
                                    source_folder,
                                    'images',
                                    image_filename)
    
            self.image_paths.append(img_path)
            self.labels.append([x_cord, y_cord, width_w, height_h])

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(img_path).convert('RGB')
        
        return label, img_path

    def __len__(self):
        return len(self.image_paths)
        
    def tackle_idxs(self, idxs):
        image_paths_temp = []
        labels_temp = []
        
        for i in idxs:
            label = self.labels[i]
            img_path = self.image_paths[i]
            
            image_paths_temp.append(img_path)
            labels_temp.append(label)
        
        combined = list(zip(image_paths_temp, labels_temp))
        random.shuffle(combined)
        self.image_paths, self.labels = map(list, zip(*combined))

In [7]:
class DenseNet121(nn.Module):
    def __init__(self):
        super().__init__()
        self.densenet = torchvision.models.densenet121(weights="IMAGENET1K_V1")

        num_features = self.densenet.classifier.in_features
        #self.densenet.classifier = nn.Linear(num_features, 1)
        
        self.densenet.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )
        
        for param in self.densenet.parameters():
            param.requires_grad = False

        to_unfreeze = [
            "features.denseblock2",
            "features.transition2",
            "features.denseblock3",
            "features.transition3",
            "features.denseblock4",
            "features.norm5",
            "classifier",
        ]
        
        for name, param in self.densenet.named_parameters():
            if any(name.startswith(layer) for layer in to_unfreeze):
                param.requires_grad = True
    
    def forward(self, x):
        return self.densenet(x)

In [8]:
class DenseNet121GradCam(nn.Module):
    def __init__(self, model):
        super().__init__()

        self.densenetGradCam = model
        self.features_conv = model.densenet.features[:11]
        # print(self.densenetGradCam)
        # print(self.features_conv)
        self.batch_norm = model.densenet.features.norm5
        self.avgpool_head = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.classifier = model.densenet.classifier
        self.sigmoid = nn.Sigmoid()
        self.gradients = None

    def activation_hook(self, grad):
        self.gradients = grad
 
    def forward(self, x):
        x = self.features_conv(x)
        h = x.register_hook(self.activation_hook)
        x = self.batch_norm(x)
        x = self.avgpool_head(x)
        x = self.flatten(x)
        x = self.classifier(x)
        
        return x
    
    def get_activation_gradients(self):
        return self.gradients
    
    def get_activations(self, x):
        return self.features_conv(x)

In [9]:
class Resnet34(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.resnet34 = torchvision.models.resnet34(weights='IMAGENET1K_V1')

        for param in self.resnet34.parameters():
            param.requires_grad = False

        to_unfreeze = ['layer3', 'layer4', 'fc']
        for name, param in self.resnet34.named_parameters():
            if any(name.startswith(layer) for layer in to_unfreeze):
                param.requires_grad = True

        num_ftrs = self.resnet34.fc.in_features
        self.resnet34.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.resnet34(x)

In [10]:
class Resnet34GradCam(nn.Module):
    def __init__(self, model):
        super().__init__()

        self.resnetGradCam = model
        self.features_conv = nn.Sequential(*list(model.resnet34.children())[:-2])

        # print(self.resnetGradCam)
        # print(self.features_conv)
        
        self.avgpool_head = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.classifier = model.resnet34.fc
        self.sigmoid = nn.Sigmoid()
        self.gradients = None

    def activation_hook(self, grad):
        self.gradients = grad
 
    def forward(self, x):
        x = self.features_conv(x)
        h = x.register_hook(self.activation_hook)
        
        x = self.avgpool_head(x)
        x = self.flatten(x)
        x = self.classifier(x)
        
        return x
    
    def get_activation_gradients(self):
        return self.gradients
    
    def get_activations(self, x):
        return self.features_conv(x)

In [11]:
class Inception_V3(nn.Module):
    def __init__(self):
        super().__init__()
        self.inception_v3 = torchvision.models.inception_v3(pretrained=True)
        
        num_features = self.inception_v3.fc.in_features
        self.inception_v3.fc = nn.Linear(num_features, 1)
        
        if self.inception_v3.aux_logits:
            num_features_aux = self.inception_v3.AuxLogits.fc.in_features
            self.inception_v3.AuxLogits.fc = nn.Linear(num_features_aux, 1)

        for name, param in self.inception_v3.named_parameters():
            param.requires_grad = False
                
        for name, param in self.inception_v3.named_parameters():
            if name.startswith('Mixed_7c') or name.startswith('fc') or name.startswith('AuxLogits.fc'):
                param.requires_grad = True
    
    def forward(self, x):
        return self.inception_v3(x)

In [12]:
class Inception_V3GradCam(nn.Module):
    def __init__(self, model):
        super().__init__()

        self.inceptionGradCam = model
        self.features_conv = nn.Sequential(
            *[
              m for m in model.inception_v3.children()
              if not isinstance(m, InceptionAux)
            ][:-3]
        )

        # print(self.inceptionGradCam)
        # print(self.features_conv)
        
        self.avgpool_head = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.classifier = model.inception_v3.fc
        self.sigmoid = nn.Sigmoid()
        self.gradients = None

    def activation_hook(self, grad):
        self.gradients = grad
 
    def forward(self, x):
        x = self.features_conv(x)
        h = x.register_hook(self.activation_hook)
        x = self.avgpool_head(x)
        x = self.flatten(x)
        x = self.classifier(x)
        
        return x
    
    def get_activation_gradients(self):
        return self.gradients
    
    def get_activations(self, x):
        return self.features_conv(x)

In [13]:
def main():
    test_dataset_Inception_V3 = SegmentedChestXRayDataset(299)
    test_loader_Inception_V3 = DataLoader(test_dataset_Inception_V3, batch_size=1, shuffle=False)
    
    test_dataset = SegmentedChestXRayDataset(224)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    test_dataset_bbox = ChestXRayDataset()
    test_loader_bbox = DataLoader(test_dataset_bbox, batch_size=1, shuffle=False)
    
    model_DenseNet121 = DenseNet121().to(device)
    model_DenseNet121.load_state_dict(torch.load("/kaggle/input/densenet121_final_model.pth/pytorch/default/1/DenseNet121_final_model.pth", map_location=device))
    model_DenseNet121.eval()
    
    gradcam_DenseNet121 = DenseNet121GradCam(model_DenseNet121)
    gradcam_DenseNet121.eval()
    
    model_Resnet34 = Resnet34().to(device)
    model_Resnet34.load_state_dict(torch.load("/kaggle/input/resnet34_final_model.pth/pytorch/default/1/Resnet34_final_model.pth", map_location=device))
    model_Resnet34.eval()
    
    gradcam_Resnet34 = Resnet34GradCam(model_Resnet34)
    gradcam_Resnet34.eval()
    
    model_Inception_V3 = Inception_V3().to(device)
    model_Inception_V3.load_state_dict(torch.load("/kaggle/input/inception_v3_final_model.pth/pytorch/default/1/Inception_V3_final_model.pth", map_location=device))
    model_Inception_V3.eval()
    
    gradcam_Inception_V3 = Inception_V3GradCam(model_Inception_V3)
    gradcam_Inception_V3.eval()

    for idx, ((image_Inception_V3, img_path_Inception_V3), (image_Resnet34_DenseNet121, img_path_Resnet34_DenseNet121), (label_bbox, img_path_bbox)) in enumerate(zip(test_loader_Inception_V3, test_loader, test_loader_bbox)):
        print(idx)
        
        image_Resnet34_DenseNet121 = image_Resnet34_DenseNet121.to(device)
        image_Inception_V3 = image_Inception_V3.to(device)
        image_Resnet34_DenseNet121.requires_grad = True
        image_Inception_V3.requires_grad = True
        
        for name_of_gradcam, gradcam, image, img_path in [("gradcam_DenseNet121", gradcam_DenseNet121, image_Resnet34_DenseNet121, img_path_Resnet34_DenseNet121), ("gradcam_Resnet34", gradcam_Resnet34, image_Resnet34_DenseNet121, img_path_Resnet34_DenseNet121), ("gradcam_Inception_V3", gradcam_Inception_V3, image_Inception_V3, img_path_Inception_V3)]:
            gradcam.zero_grad(
            logits = gradcam(image)
            logits[0, 0].backward()
    
            activations = gradcam.get_activations(image).cpu().detach()
            gradients = gradcam.get_activation_gradients().cpu()
            pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    
            for i in range(activations.shape[1]):
                activations[:, i, :, :] *= pooled_gradients[i]
    
            heatmap = torch.sum(activations, dim=1).squeeze()
            heatmap = torch.clamp(heatmap, min=0)
            heatmap /= torch.max(heatmap)
            heatmap = heatmap.numpy()
    
            image_original = cv2.imread(img_path[0])
            heatmap_resized = cv2.resize(heatmap, (image_original.shape[1], image_original.shape[0]))
            heatmap_resized_uint8 = np.uint8(255 * heatmap_resized)
            heatmap_color = cv2.applyColorMap(heatmap_resized_uint8, cv2.COLORMAP_JET)
            superimposed_img = heatmap_color * 0.4 + image_original
            superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
    
            neighborhood_size = 15
            threshold_val = 0.1
    
            heatmap_norm = heatmap_resized_uint8.copy()
    
            data_max = maximum_filter(heatmap_norm, size=neighborhood_size)
            maxima = (heatmap_norm == data_max)
            data_min = minimum_filter(heatmap_norm, size=neighborhood_size)
            diff = ((data_max - data_min) > threshold_val)
            maxima[diff == 0] = 0
    
            for _ in range(5):
                maxima = binary_dilation(maxima)
    
            labeled, num_objects = ndimage.label(maxima)
            centers = ndimage.center_of_mass(heatmap_norm, labeled, range(1, num_objects+1))
    
            scale_factor = image_original.shape[1] / 224
            w_k = int(122 * scale_factor)
            h_k = int(132 * scale_factor)
            
            for center in centers:
                y, x = center
                if heatmap_norm[int(y), int(x)] > np.max(heatmap_norm) * 0.9:
                    left = int(max(x - w_k/2, 0))
                    upper = int(max(y - h_k/2, 0))
                    right = int(min(left + w_k, image_original.shape[1]))
                    lower = int(min(upper + h_k, image_original.shape[0]))
                    left = int(left)
                    right = int(right)
                    upper = int(upper)
                    lower = int(lower)
    
                    cv2.rectangle(superimposed_img, (left, upper), (right, lower), (0, 255, 0), 2)
    
            cv2.imwrite(f"{name_of_gradcam}_{idx}.jpg", superimposed_img)

        original_img_bbox = cv2.imread(img_path_bbox[0])
        if original_img_bbox is not None:
            x, y, w, h = label_bbox
            x = int(x.item())
            y = int(y.item())
            w = int(w.item())
            h = int(h.item())
            print(x)
            print(y)
            print(w)
            print(h)
            cv2.rectangle(original_img_bbox, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            cv2.imwrite(f"orig_{idx}.jpg", original_img_bbox)
        else:
            print(f"Failed to load image: {img_path_bbox[0]}")

if __name__ == '__main__':
    main()

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 163MB/s]
/tmp/ipykernel_31/2002097209.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the

0
667
276
122
150
1
899
690
53
58
2
741
687
52
48
3


/tmp/ipykernel_31/2002097209.py:59: RuntimeWarning: invalid value encountered in cast
  heatmap_resized_uint8 = np.uint8(255 * heatmap_resized)


351
747
35
33
4
118
483
85
101
5
739
391
52
61
6
696
319
54
57
7
350
458
92
140
8
640
692
87
79
9
68
649
48
44
10
655
256
82
80
11
281
305
102
127
12
190
607
63
69
13
237
332
52
48
14
199
728
102
115
15
267
293
79
78
16
623
182
66
58
17
614
808
74
75
18
763
389
74
86
19
819
345
52
57
20
770
472
45
46
21
241
642
66
63
22
825
309
72
115
23
668
652
121
79
24
711
357
58
47
25
175
445
74
66
26
626
385
32
26
27
159
437
100
68
28
552
190
68
72
29
701
482
80
74
30
150
435
81
70
31
695
319
100
88
32
792
417
55
53
33
266
550
58
57
34
229
623
74
63
35
76
602
88
83
36
827
639
50
57
37
399
500
94
73
38
783
558
48
45
39
796
603
50
66
40
364
265
48
46
41
746
377
121
105
42
773
682
70
74
43
678
340
47
35
44
300
284
58
76
45
282
257
98
110
46
327
381
68
73
47
905
824
47
47
48
796
327
78
59
49
301
266
62
49
50
237
227
75
70
51
805
360
55
57
52
744
351
70
73
53
787
271
70
76
54
873
615
28
33
55
212
490
79
60
56
436
273
43
44
57
229
477
60
62
58
190
341
65
72
59
145
579
76
58
60
748
417
75
52
61
171
192
9

In [14]:
import os
import zipfile

OUT_BASE = "/kaggle/working/"
zip_path = os.path.join(OUT_BASE, "gradcam_images.zip")

with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for fname in os.listdir(OUT_BASE):
        full = os.path.join(OUT_BASE, fname)
        if os.path.isfile(full) and fname.lower().endswith(".jpg"):
            zf.write(full, arcname=fname)

print(f"Created ZIP archive at {zip_path}")

Created ZIP archive at /kaggle/working/gradcam_images.zip
